In [1]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: '../requirements.txt'

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# Please choose LLM")
    with gr.Row():
        select_llm = gr.Dropdown(
            ["GPT", "Alpaca Lora"], value="GPT", 
            label="It's a lot better to use GPT but if you don't want to get openai api key, you can use Alpaca Lora.", 
            interactive=True
        )
    with gr.Row(equal_height=True):
        with gr.Column(scale=9):
            openai_api_key_or_gradio_url_box = gr.Textbox(
                show_label=False, container=False, interactive= True,
                placeholder="Enter OpenAI API Key", 
                info="How to get OpenAI API Key...")
        with gr.Column(min_width=70, scale=1):
            submit_button = gr.Button(value="Submit")
        
    def set_llm(selected_llm: str) -> None:
        """
        Set llm and update message in textbox accordingly.
        
        :param selected_llm: user selected llm
        """
        global llm
        llm = selected_llm
        if llm == "GPT":
            return gr.update(
                value="", 
                placeholder="Enter OpenAI API Key", 
                info="How to get OpenAI API Key...")
        else:
            return gr.update(
                value="", 
                placeholder="Enter Gradio URL", 
                info="How to get Gradio URL...")
            
    def set_openai_api_key_or_gradio_url(openai_api_key_or_gradio_url_input: str) -> None:
        """
        Set openai_api_key_or_gradio_url.
        
        :param openai_api_key_or_gradio_url_input: user input for openai_api_key_or_gradio_url
        """
        global openai_api_key_or_gradio_url
        openai_api_key_or_gradio_url = openai_api_key_or_gradio_url_input
    
    select_llm.input(
        fn=set_llm, inputs=select_llm, outputs=openai_api_key_or_gradio_url_box)
    
    openai_api_key_or_gradio_url_box.submit(
        fn=set_openai_api_key_or_gradio_url, inputs=openai_api_key_or_gradio_url_box)
    
    submit_button.click(
        fn=set_openai_api_key_or_gradio_url, inputs=openai_api_key_or_gradio_url_box)
        
if __name__ == "__main__":
    demo.launch()

c:\Users\chanm\Documents\GitHub\llm-convrec\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [2]:
import gradio as gr
import yaml
import time
from conv_rec_system import ConvRecSystem
from domain_specific.classes.restaurants.location_constraint_merger import LocationConstraintMerger

with open("system_config.yaml") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
config['LLM'] = llm

user_merge_constraint_objects = [LocationConstraintMerger()]

conv_rec_system = ConvRecSystem(
        config, user_merge_constraint_objects, openai_api_key_or_gradio_url, user_interface_str="demo")

with open("demo.css", "r", encoding="utf-8") as f:
    demo_css = f.read()

with gr.Blocks(css=demo_css) as demo:
    gr.Markdown("# LLM Convrec")
    history = gr.State([[None, conv_rec_system.init_msg]])
    with gr.Row():
        chatbot = gr.Chatbot(
            value=[[None, conv_rec_system.init_msg]], show_label=False, elem_id="llm_conv_rec")
    with gr.Row(equal_height=True):
        with gr.Column(scale=8):
            user_input = gr.Textbox(show_label=False, placeholder="Enter text", container=False)
        with gr.Column(min_width=70, scale=1):
            send_button = gr.Button(value="Send")
        with gr.Column(min_width=70, scale=3):
            new_conv_button = gr.Button(value="New Conversation")

    def display_user_input(user_message, chatbot, history) -> tuple[gr.Textbox, gr.Chatbot, gr.State]:
        """
        Display user input.
        
        :param user_message: user input
        :param chatbot: chatbot that display chat history
        :param history: chat history
        :return: a tuple of textbox,chatbot, and state that are updated
        """
        chatbot[-1][1] = history[-1][1]
        return "", chatbot + [[user_message, None]], history + [[user_message, None]]

    def display_recommender_response(chatbot, history) -> tuple[gr.Chatbot, gr.State]:
        """
        Display recommender's response.
        
        :param chatbot: chatbot that display chat history
        :param history: chat history
        :return: a tuple of chatbot and state that are updated
        """
        bot_message = conv_rec_system.get_response(chatbot[-1][0])
        history[-1][1] = bot_message
        chatbot[-1][1] = ""
        for character in bot_message:
            chatbot[-1][1] += character
            time.sleep(0.015)
            yield chatbot, history

    def reset_state() -> tuple[gr.Textbox, gr.Chatbot, gr.State]:
        """
        Reset state.
        
        :return: a tuple of textbox, chatbot, and state that are reset
        """
        conv_rec_system.dialogue_manager.state_manager.reset_state()
        return "", [[None, conv_rec_system.init_msg]], [[None, conv_rec_system.init_msg]]
    
    user_input.submit(
        fn=display_user_input, inputs=[user_input, chatbot, history], 
        outputs=[user_input, chatbot, history], queue=True).then(
            fn=display_recommender_response, inputs=[chatbot, history], outputs=[chatbot, history])
        
    send_button.click(
        fn=display_user_input, inputs=[user_input, chatbot, history], 
        outputs=[user_input, chatbot, history], queue=True).then(
            fn=display_recommender_response, inputs=[chatbot, history], outputs=[chatbot, history])
        
    new_conv_button.click(
        fn=reset_state, outputs=[user_input, chatbot, history], queue=True)


if __name__ == "__main__":
    demo.queue()
    demo.launch()
    

Exception: The provided Gradio URL is invalid. Please input a correct url and retry.